In [26]:
import requests
from dotenv import load_dotenv
from time import sleep
import os
from typing import List

### Step 1: Load token into headers

In [2]:
# Load environment variables from the .env file
load_dotenv("token.env")

True

In [3]:
hubspot_bearer_token = os.getenv("HUBSPOT_BEARER_TOKEN")
lexoffice_api = os.getenv("LEXOFFICE_API")

In [4]:
# Hubspot headers
hubspot_headers = {"Authorization": f"Bearer {hubspot_bearer_token}"}

In [5]:
# LexOffice headers
lexoffice_headers = {"Authorization": f"Bearer {lexoffice_api}"}

### Step 2: Call all WON Deals in Hubspot using Pagination

In [31]:
### This list are the IDs assigned to 'WON' dealstage in the sales pipeline
won_dealstages = ["13546489", "80458335", "257294988", "94979183"]

In [32]:
# Step 2: API Request with Pagination
url = "https://api.hubapi.com/crm/v3/objects/deals"
params = {
    "limit": 50,
    "properties": ["dealname", "dealstage", "hs_lastmodifieddate", "createdate"],
    "propertiesWithHistory": ["dealstage"],
}

after = None
has_more = True

recently_became_win = []

while has_more:
    if after:
        params["after"] = after

    response = requests.get(url, headers=hubspot_headers, params=params)
    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        break
    data = response.json()

    # process deals
    for deal in data["results"]:
        properties = deal["properties"]
        if (
            "(Import" not in properties["dealname"]
            and "(Upgrade" not in properties["dealname"]
        ):
            # convert last modified date strings to timestamps
            hs_lastmodifieddate_str = properties["hs_lastmodifieddate"]

            dealstage_history = []
            ### dealstage history contains all the dealstage of the deal with its respective timestamps
            if (
                "propertiesWithHistory" in deal
                and "dealstage" in deal["propertiesWithHistory"]
            ):
                dealstage_history = deal["propertiesWithHistory"]["dealstage"]

                for version in dealstage_history:
                    ### this if else function looks for deal with dealstage recently transitioned to 'WIN'
                    if version["value"] in won_dealstages:
                        recently_became_win.append(deal)
                        break

    # paginating
    has_more = "paging" in data and "next" in data["paging"]
    if has_more:
        after = data["paging"]["next"]["after"]
    else:
        after = None

    sleep(0.5)

In [ ]:
### Print test
print("\n All Won Deals are:")
for deal in recently_became_win:
    properties = deal["properties"]
    print(f"- Deal Name: {properties['dealname']}")

In [42]:
### Save Deals into a list of only dealname --> For comparision with LexOffice
hubspot_dealname = []
for i in recently_became_win:
    dealname = i["properties"]["dealname"]
    hubspot_dealname.append(dealname)

### Step 3: Paginate through all Recurring Templates in Lexoffice (Serienrechnungen) and get the 'name' of all 'Rechnung' into a list for comparision later

In [50]:
def get_contact_name_lexoffice(page_number: int) -> List[str]:
    """
    Call for all Serienvorlagen in LexOffice and return a list of names (from contacts) to compare with Hubspot later on.

    :param page_number: The page number to retrieve from the LexOffice API.
    :return: A list of contact names (strings).
    """
    rec_template_get_url = f"https://api.lexoffice.io/v1/recurring-templates?page={page_number}&size=250&sort=createdDate,DESC"
    response = requests.get(rec_template_get_url, headers=lexoffice_headers)

    if response.status_code == 200:
        print("Successfully retrieved Serienvorlagen")
        json_response = response.json()  # Parse JSON once
        content = json_response.get(
            "content", []
        )  # Safely get 'content', defaulting to an empty list if not found
        serien_vorlage_namen = []
        if content:
            for item in content:
                name = item.get("address", {}).get(
                    "name", ""
                )  # Safely get 'name' from 'address'
                if (
                    name and name not in serien_vorlage_namen
                ):  # Only add non-empty names + Avoid duplicates
                    serien_vorlage_namen.append(name)

        # Return either a list of names or just an empty list
        return serien_vorlage_namen
    else:
        print(
            f"Error. Status code is {response.status_code}. Response: {response.text}"
        )
        return []  # Return an empty list in case of error

In [51]:
serien_vorlage_namen = get_contact_name_lexoffice(0)

Successfully retrieved Serienvorlagen


In [52]:
not_in_serienvorlagen = []

In [53]:
for name in hubspot_dealname:
    if name not in serien_vorlage_namen:
        not_in_serienvorlagen.append(name)

In [54]:
print(len(hubspot_dealname))
print(len(not_in_serienvorlagen))

136
75


In [55]:
print(not_in_serienvorlagen)

['NATULIS Group AG', 'Soravia Investment Holding GmbH', 'TSC Real Estate GmbH', 'CILON GmbH', 'Seyband Gruppe (User: 3)', 'Heimstaden Germany GmbH', 'Cube Real Estate GmbH', 'OFB Projektentwicklung GmbH', 'Neutecta Grundinvest AG', 'Schettler Immobilien Gruppe', 'Peper & Söhne Projekt GmbH', '4BRICKS', 'BBG Invest', 'Graubner Projektentwicklung GmbH & Co. KG', 'Alpine Immobilien GmbH', 'S&P Commercial Development GmbH', 'INDUSTRIA WOHNEN GmbH', 'POLIS Immobilien AG', 'Georg Schewe GmbH & Co. KG', 'WERTCONCEPT Investment Group GmbH', 'PREIG AG', 'CV Real Estate AG', 'Pegasus Capital Partners GmbH', 'Tecklenburg Projektentwicklungs GmbH', 'Real Trade', 'weisenburger invest GmbH', 'Engelhardt Group', 'DOWNTOWN Development GmbH', 'F4 Immobilien (User: 5)', 'S&P Grund Invest GmbH', 'Soravia Deutschland (User: 5)', 'G-PARX Real Estate GmbH', 'IKEA Verwaltungs-GmbH', 'BUDIRO', 'T&T Grundbesitz', 'STONEHEDGE Projektentwicklungsgesellschaft mbH', 'SONAR Real Estate GmbH', 'Laborgh Investment Gm